### Student Information

#### **Name:** Victor D. Lopez

#### **Student ID:** 110062426

#### **GitHub ID:** vlopez0392

---

### Instructions

1. First: do the **take home** exercises in the [DM2023-Lab1-Master](https://github.com/fjrialdnc0615/DM2023-Lab1-Master). You may need to copy some cells from the Lab notebook to this notebook. __This part is worth 20% of your grade.__


2. Second: follow the same process from the [DM2023-Lab1-Master](https://github.com/fjrialdnc0615/DM2023-Lab1-Master) on **the new dataset**. You don't need to explain all details as we did (some **minimal comments** explaining your code are useful though).  __This part is worth 30% of your grade.__
    - Download the [the new dataset](https://archive.ics.uci.edu/ml/datasets/Sentiment+Labelled+Sentences#). The dataset contains a `sentence` and `score` label. Read the specificiations of the dataset for details. You need to combine three labeled datasets into one file for your data preparation part.
    - You are allowed to use and modify the `helper` functions in the folder of the first lab session (notice they may need modification) or create your own.


3. Third: please attempt the following tasks on **the new dataset**. __This part is worth 30% of your grade.__
    - Generate meaningful **new data visualizations**. Refer to online resources and the Data Mining textbook for inspiration and ideas.
    - Generate **TF-IDF features** from the tokens of each text. This will generating a document matrix, however, the weights will be computed differently (using the TF-IDF value of each word per document as opposed to the word frequency). Refer to this Sciki-learn [guide](http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html) .
    - Implement a simple **Naive Bayes classifier** that automatically classifies the records into their categories. Use both the TF-IDF features and word frequency features to build two seperate classifiers. Comment on the differences.  Refer to this [article](https://hub.packtpub.com/implementing-3-naive-bayes-classifiers-in-scikit-learn/).


4. Fourth: In the lab, we applied each step really quickly just to illustrate how to work with your dataset. There are somethings that are not ideal or the most efficient/meaningful. Each dataset can be habdled differently as well. What are those inefficent parts you noticed? How can you improve the Data preprocessing for these specific datasets? __This part is worth 10% of your grade.__


5. Fifth: It's hard for us to follow if your code is messy, so please **tidy up your notebook** and **add minimal comments where needed**. __This part is worth 10% of your grade.__


You can submit your homework following these guidelines: [Git Intro & How to hand your homework](https://github.com/fjrialdnc0615/DM2023-Lab1-Master/blob/main/Git%20Intro%20%26%20How%20to%20hand%20your%20homework.ipynb). Make sure to commit and save your changes to your repository __BEFORE the deadline (October 27th 11:59 pm, Thursday)__.

## **PART I: Take-Home Exercises**

In [ ]:
# TEST necessary for when working with external scripts
%load_ext autoreload
%autoreload 2

# Imports
from sklearn.datasets import fetch_20newsgroups
import pandas as pd
import helpers.data_mining_helpers as dmh

categories = ['alt.atheism', 'soc.religion.christian', 'comp.graphics', 'sci.med']
twenty_train = fetch_20newsgroups(subset='train', categories=categories, shuffle=True, random_state=42)

## Construct dataframe from a list
X = pd.DataFrame.from_records(dmh.format_rows(twenty_train), columns = ["text"]);

### Adding category and category names
X["category"] = twenty_train.target;
X["category_name"] = X.category.apply(lambda t: dmh.format_labels(t, twenty_train));



### **Exercise 2**

Experiment with other querying techniques using pandas dataframes.

#### **SOLUTION:**

For instance, we may use Selection by callable using a callable anonymous function with the dataframe as argument which returns valid output for indexing. We solve Exercise 3 in an alternative fashion with this query technique as follows:

In [10]:
###EXERCISE 2

X.loc[lambda df: df['category'] == 2][::10][:5]

,text,category,category_name
7,From: aldridge@netcom.com (Jacquelin Aldridge)...,2,sci.med
49,From: jimj@contractor.EBay.Sun.COM (Jim Jones)...,2,sci.med
82,From: jason@ab20.larc.nasa.gov (Jason Austin) ...,2,sci.med
118,From: rogers@calamari.hi.com (Andrew Rogers) S...,2,sci.med
142,From: lady@uhunix.uhcc.Hawaii.Edu (Lee Lady) S...,2,sci.med


### **Exercise 5**

When we are working with extremely large data, it's difficult to check records one by one (as we have been doing so far). And also, we don't even know what kind of missing values we are facing. Thus, "debugging" skills get sharper as we spend more time solving bugs. Let's focus on a different method to check for missing values and the kinds of missing values you may encounter. It's not easy to check for missing values as you will find out in a minute.

Please check the data and the process below, describe what you observe and why it happened.


In [8]:
import numpy as np

NA_dict = [{ 'id': 'A', 'missing_example': np.nan },
           { 'id': 'B'                    },
           { 'id': 'C', 'missing_example': 'NaN'  },
           { 'id': 'D', 'missing_example': 'None' },
           { 'id': 'E', 'missing_example':  None  },
           { 'id': 'F', 'missing_example': ''     }]

NA_df = pd.DataFrame(NA_dict, columns = ['id','missing_example'])
NA_df

,id,missing_example
0,A,NaN
1,B,NaN
2,C,NaN
3,D,None
4,E,None
5,F,


In [9]:
NA_df['missing_example'].isnull()


0     True
1     True
2    False
3    False
4     True
5    False
Name: missing_example, dtype: bool

#### **SOLUTION:**

The solution is simple. The records corresponding to records 2,3, and 5 are not null values. They are strings. Regardless if they are empty strings or have values such as 'None' or 'NaN', they cannot be considered as missing values.  

According to the documentation, for scalar or array-like objects, only NaN in numeric arrays, None or NaN in object arrays can be considered as missing values. In addition, datetimelike values are with NaT can be considered missing values.

Thus, *isnull()* technically doesn't fail since it returns a correct value according to the given input. If we were to use data which considered string values such as 'None', 'null', 'NaN' or ' ' as missing values, we should first parse these values into the appropriate undefined values before using *isnull()*.

## **PART II: Sentiment Labelled Sentences Dataset**

### **II.1 Creating our Dataset**

First, we import the necessary modules needed to create the dataset. Some of them were imported previously in this noteboook so make sure to execute the cells above first. We create some lists with useful data such as text file names and website names.

In [59]:
###Imports
import os
import numpy as np
from itertools import chain

###TXT Filenames
txt_files = ["imdb_labelled.txt", "amazon_cells_labelled.txt", "yelp_labelled.txt"];

### Website names and categories
websites = [("imdb",2), ("amazon",3), ("yelp",4)];

The folllowing helper functions read the text files contained in the "sentiment_labelled_sentences" directory (our raw dataset) and parse them into lists containing the text data and targets (either positive or negative reviews).  

In [67]:
###HELPER FUNCTIONS
###Get text (data) and review (target) from list of file lines
def getTextAndReview(file_lines):
  curr_list = [];
  data = [];
  targets = [];

  for st in file_lines:
    curr_list = st.split("\t");
    data.append(str(curr_list[0].strip()));         ##Remove trailing whitespace since its meaningless
    targets.append(int(curr_list[1].rstrip('\n'))); ##Remove the newline character and store as an int

  return [data, targets];

###Parse text file into text (string) and review (int) helper function
def parseTXT(txt_files):
  currFilePath = '';
  file_lines = [];
  data = [];
  targets = []

  ###Open and parse each file
  for file_name in txt_files:
    ##Open the file and read it
    currFilePath = os.path.join(os.getcwd(),"sentiment_labelled_sentences",file_name);
    f = open(currFilePath,'r');
    file_lines = f.readlines();

    ## Process the file and append to data and target lists
    [d, t] = getTextAndReview(file_lines = file_lines);
    data.append(d);
    targets.append(t);

    ## Close the current file
    f.close()

  data = list(chain.from_iterable(data));
  targets = list(chain.from_iterable(targets));
  return [data,targets]

According to the website where we extracted the dataset, there are a total of 3000 review instances among the three websites (imdb, amazon and yelp). For each website there are a total of 1000 reviews. We first show that the length of the returned lists is correct along with the first 5 text data and target instances:

In [63]:
### Verifying correct length of data and target lists
[data, targets] = parseTXT(txt_files);
print(len(data));
print(len(targets));

3000
3000


In [66]:
###First 5 data records
data[0:5]

['A very, very, very slow-moving, aimless movie about a distressed, drifting young man.',
 'Not sure who was more lost - the flat characters or the audience, nearly half of whom walked out.',
 'Attempting artiness with black & white and clever camera angles, the movie disappointed - became even more ridiculous - as the acting was poor and the plot and lines almost non-existent.',
 'Very little music or anything to speak of.',
 'The best scene in the movie was when Gerardo is trying to find a song that keeps running through his head.']

In [62]:
targets[0:5]

[0, 0, 0, 0, 1]

Next, similar to the category_name column in the lab tutorial we may add the website name where the review came from along with a numerical representation or index of the website. For this, we use the *websites* list of tuples we declared above. Since we know we processed the text files in the order shown in the *txt_files* list, we create two lists of size 3000 with 1000 instances of each website name and index.

In [89]:
website_names = [];
website_index = [];
length = 1000;

for tup in websites:
  website_names.append([tup[0]]*length);
  website_index.append([tup[1]]*length);

website_names = list(chain.from_iterable(website_names));
website_index = list(chain.from_iterable(website_index));

### Verify by printing length of the lists and that the first 5 elements of each website is in the correct index (0:5,1000:1005,2000:2005)
print(len(website_names));
print(len(website_index));
for k in range(0,3): print(website_names[k*length:k*length+5], website_index[k*length:k*length+5]);

3000
3000
['imdb', 'imdb', 'imdb', 'imdb', 'imdb'] [2, 2, 2, 2, 2]
['amazon', 'amazon', 'amazon', 'amazon', 'amazon'] [3, 3, 3, 3, 3]
['yelp', 'yelp', 'yelp', 'yelp', 'yelp'] [4, 4, 4, 4, 4]


Finally, from or lists, we create a Pandas dataframe and display some reviews:

In [108]:
DF = pd.DataFrame(np.column_stack([data, website_names, website_index, targets]),
                               columns=['text','website_name', 'website_index', 'target_review'])

In [109]:
DF[:10]

,text,website_name,website_index,target_review
0,"A very, very, very slow-moving, aimless movie ...",imdb,2,0
1,Not sure who was more lost - the flat characte...,imdb,2,0
2,Attempting artiness with black & white and cle...,imdb,2,0
3,Very little music or anything to speak of.,imdb,2,0
4,The best scene in the movie was when Gerardo i...,imdb,2,1
5,"The rest of the movie lacks art, charm, meanin...",imdb,2,0
6,Wasted two hours.,imdb,2,0
7,Saw the movie today and thought it was a good ...,imdb,2,1
8,A bit predictable.,imdb,2,0
9,Loved the casting of Jimmy Buffet as the scien...,imdb,2,1


In [110]:
DF[1000:1010]

,text,website_name,website_index,target_review
1000,So there is no way for me to plug it in here i...,amazon,3,0
1001,"Good case, Excellent value.",amazon,3,1
1002,Great for the jawbone.,amazon,3,1
1003,Tied to charger for conversations lasting more...,amazon,3,0
1004,The mic is great.,amazon,3,1
1005,I have to jiggle the plug to get it to line up...,amazon,3,0
1006,If you have several dozen or several hundred c...,amazon,3,0
1007,If you are Razr owner...you must have this!,amazon,3,1
1008,"Needless to say, I wasted my money.",amazon,3,0
1009,What a waste of money and time!.,amazon,3,0


In [111]:
DF[2000:2010]

,text,website_name,website_index,target_review
2000,Wow... Loved this place.,yelp,4,1
2001,Crust is not good.,yelp,4,0
2002,Not tasty and the texture was just nasty.,yelp,4,0
2003,Stopped by during the late May bank holiday of...,yelp,4,1
2004,The selection on the menu was great and so wer...,yelp,4,1
2005,Now I am getting angry and I want my damn pho.,yelp,4,0
2006,Honeslty it didn't taste THAT fresh.),yelp,4,0
2007,The potatoes were like rubber and you could te...,yelp,4,0
2008,The fries were great too.,yelp,4,1
2009,A great touch.,yelp,4,1
